In [1]:
import sys, os
from tqdm import tqdm
import torch
sys.path.append("./Torchvision/src")
from submission import model_submission, get_test_outputs, select_bboxes, format_results
from models import get_model_detection
sys.path.append("./Ensemble/ensemble-objdet/")
from legacy_ensemble import ensemble_legacy

### First:

- Check the selection. Make sure we select highest scores: argsort on the scores.

### Soumettre:
 - base .7
 - nms_all(base, iou_thr=.5) .7
 - legacy_ensemble(results, iou_thr=.5) .7
 - nms_ensemble(results, iou_thr=.5) .7

In [2]:
def convert_eval_format(results):
    out_results = {}
    for k in results:
        out_results[k] = {
            "dt_lbl"   : results[k]["labels"].cpu().numpy(),
            "dt_bbox"  : results[k]["boxes"].cpu().numpy(),
            "dt_score" : results[k]["scores"].cpu().numpy(),
            "gt_lbl"   : results[k]["labels"].cpu().numpy(),
            "gt_bbox"  : results[k]["boxes"].cpu().numpy(),
        }
    return out_results

def convert_submit_results(results):
    out_results = {}
    for k in results:
        out_results[k] = {
            "labels" : torch.from_numpy(results[k]["dt_lbl"]),
            "boxes"  : torch.from_numpy(results[k]["dt_bbox"]),
            "scores" : torch.from_numpy(results[k]["dt_score"])
        }
    return out_results

In [68]:
def extract_sub_file(results, thr=.65, maxbbox=5, name="default"):
    print(id(results))
    # Init path
    path = f"/home/tristan/road_project/workspace/Damage Detection/submissions/{name}"
    assert not os.path.isfile(path)
    
    # Filter results
    filtered_results = {}
    for key, anno in tqdm(results.items()):
        filtered_results[key] = select_bboxes(anno, thr, maxbbox)
    
    #Format string
    text_results = format_results(filtered_results, sep=",")

    # Write results
    with open(path, "w") as f:
        f.write(text_results)

In [79]:
import numpy as np
from evaluation import iou_fn

def eps(x):
    return 10**-6 * np.random.randn(*x.shape)

def nms(result, iou_thr = .1):
    bbox = result["dt_bbox"]
    scores = result["dt_score"] 
    scores+= eps(scores)
    msk = np.ones_like(scores) > 0
    a,b = np.where(iou_fn(bbox, bbox) > iou_thr)
    a,b = a[a!=b], b[a!=b]
    msk_idx = b[scores[a] > scores[b]]
    msk[msk_idx] = False
    return  {
        "gt_lbl"  : result["gt_lbl"],
        "gt_bbox" : result["gt_bbox"],
        "dt_bbox" : result["dt_bbox"][msk],
        "dt_score": result["dt_score"][msk],
        "dt_lbl"  : result["dt_lbl"][msk],
    }

def nms_all(results, iou_thr = .1):
    return {k:nms(v, iou_thr) for k,v in results.items()}

In [3]:
batch_size = 2
num_workers = 1
gpu_id = 1
device = f"cuda:{gpu_id}"
mode = "class"

In [4]:
modelpaths = {
    "base":f"./Torchvision/Checkpoints/Class types/{mode}/epoch_9",
    "aug0":f"./Torchvision/Checkpoints/augmentation/{mode}/google_0/epoch_11",
    "aug1":f"./Torchvision/Checkpoints/augmentation/{mode}/google_1/epoch_11"
}

In [5]:
# Extract results
results = {}
for k,model_path in modelpaths.items():
    model = get_model_detection(5).to(device)
    model.load_state_dict(torch.load(model_path))
    res = get_test_outputs(model)
    results[k]=convert_eval_format(res)

  0%|          | 0/2631 [00:00<?, ?it/s]/home/tristan/anaconda3/envs/detectron/lib/python3.8/site-packages/torch/nn/functional.py:2854: UserWarning: The default behavior for interpolate/upsample with float scale_factor will change in 1.6.0 to align with other frameworks/libraries, and use scale_factor directly, instead of relying on the computed output size. If you wish to keep the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn("The default behavior for interpolate/upsample with float scale_factor will change "
/opt/conda/conda-bld/pytorch_1587428207430/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of nonzero is deprecated:
	nonzero(Tensor input, *, Tensor out)
Consider using one of the following signatures instead:
	nonzero(Tensor input, *, bool as_tuple)
100%|██████████| 2631/2631 [05:54<00:00,  7.42it/s]


### Legacy ensemble

In [81]:
ensemble_results = ensemble_legacy(results, iou_thr=.5)
ensemble_results = convert_submit_results(ensemble_results)
extract_sub_file(ensemble_results, name="legacy_ensemble_iou=.01")

100%|██████████| 2631/2631 [00:00<00:00, 21176.24it/s]


,


### baseline

In [ ]:
extract_sub_file(results["base"], name="baseline")

### baseline nms

In [82]:
nms_results = nms_all(results["base"], iou_thr=.5)
nms_results = convert_submit_results(nms_results)
extract_sub_file(nms_results, name="nms_baseline_ious")

100%|██████████| 2631/2631 [00:00<00:00, 7802.76it/s]

,


### OLD

In [ ]:
# Filter results
thr = .63
maxbbox = 5
filtered_results = {}
lbls = []
for key, anno in tqdm(ensemble_results.items()):
    filtered_results[key] = select_bboxes(anno, thr, maxbbox)
    lbls.extend(filtered_results[key]["labels"].cpu().numpy().tolist())
    
# Format
text_results = format_results(filtered_results, sep=",")

#name = f"baseline_submission_{mode}_thr{thr}_maxbb{maxbbox}"
path = f"/home/tristan/road_project/workspace/Damage Detection/submissions/{name}"
assert not os.path.isfile(path)
with open(path, "w") as f:
    f.write(text_results)